In [ ]:
import keras
from keras.models import Sequential
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception, preprocess_input
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
import cv2, numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Пути к наборам данных
train_dir = '/content/drive/MyDrive/Keras/train'
test_dir = '/content/drive/MyDrive/Keras/validation' 
# Размер изображений
img_width, img_height = 500, 500
# Размер мини-выборки
batch_size = 60
# Кол-во изображений для обучения
nb_train_samples = 1034
# Кол-во изображений для теста
nb_test_samples = 128

In [ ]:
# Создание генератора изображений
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1034 images belonging to 3 classes.
Found 128 images belonging to 3 classes.


In [ ]:
# Создание экземпляра модели сети Xception
xception = Xception(
    input_shape=(img_width, img_height, 3),
    include_top=False,
    weights="imagenet"
)
# Сверточную часть сети обучать не нужно
xception.trainable = False

xception.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 500, 500, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 249, 249, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 249, 249, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 249, 249, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
# Создание модели составной сети (Составная сеть)
model = Sequential()
# Добавляем сверточные слои
model.add(xception)
# Преобразуем двумерный массив MobileNet в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(3, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 16, 16, 2048)      20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 524288)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               134217984 
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 771       
Total params: 155,080,235
Trainable params: 134,218,755
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='SGD', 
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=12,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/12
18/18 [==============================] - 35s 2s/step - loss: 4.5509 - accuracy: 0.4280 - val_loss: 0.8160 - val_accuracy: 0.6455
Epoch 2/12
18/18 [==============================] - 25s 1s/step - loss: 0.7544 - accuracy: 0.6459 - val_loss: 0.5674 - val_accuracy: 0.7000
Epoch 3/12
18/18 [==============================] - 25s 1s/step - loss: 0.5875 - accuracy: 0.7670 - val_loss: 0.9175 - val_accuracy: 0.6364
Epoch 4/12
18/18 [==============================] - 25s 1s/step - loss: 0.5607 - accuracy: 0.7784 - val_loss: 0.6226 - val_accuracy: 0.7636
Epoch 5/12
18/18 [==============================] - 25s 1s/step - loss: 0.4832 - accuracy: 0.8042 - val_loss: 0.3808 - val_accuracy: 0.8091
Epoch 6/12
18/18 [==============================] - 25s 1s/step - loss: 0.3418 - accuracy: 0.8645 - val_loss: 0.4703 - val_accuracy: 0.7636
Epoch 7/12
18/18 [==============================] - 25s 1s/step - loss: 0.2959 - accuracy: 0.8862 - val_loss: 0.3346 - val_accuracy: 0.8727
Epoch 8/12
18/18 [==

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print('Точность работы на тестовых данных: %.2f%%' % (scores[1]*100))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1948: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Точность работы на тестовых данных: 88.18%
